# Generar 'X' e 'y' para todos los experimentos
<p>Como datos para las X se toma la media de los 17 canales de:</p>
<ul>
    <li>X_1: PSD de la banda delta (1~4Hz)</li>
    <li>X_2: PSD de la banda theta (4~8Hz)</li>
    <li>X_3: PSD de la banda alpha (8~14Hz)</li>
    <li>X_4: PSD de la banda beta (14~31Hz)</li>
    <li>X_5: PSD de la banda gamma (31~50 Hz)</li>
</ul>

In [ ]:
import mne
from mne.externals.pymatreader import read_mat

from read import read_features

import numpy as np

import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
labels = {
    'leve': 0,
    'moderada': 1,
    'severa': 2
}

def parse_y(y, lim_leve=0.075, lim_moderada=0.15):
    def classify(val):
        if (val <= lim_leve):
            return labels['leve']
        if (val <= lim_moderada):
            return labels['moderada']
        return labels['severa']
    
    len_y = len(y)
    classified = np.zeros(len_y, dtype=int)
    
    for i in range(len_y):
        classified[i] = classify(y[i])
        
    return classified

In [ ]:
# Las que mejores graficas tienen
# features_files = ['4_20151105_noon.mat', '4_20151107_noon.mat', '5_20141108_noon.mat','12_20150928_noon.mat','14_20151014_night.mat', '18_20150926_noon.mat', '21_20151016_noon.mat']

# features_files = ['4_20151105_noon.mat', '5_20141108_noon.mat', '21_20151016_noon.mat']

features_files = ['1_20151124_noon_2.csv', '2_20151106_noon.csv', '3_20151024_noon.csv','4_20151105_noon.csv', '4_20151107_noon.csv',
            '5_20141108_noon.csv', '5_20151012_night.csv', '6_20151121_noon.csv','7_20151015_night.csv', '8_20151022_noon.csv', 
            '9_20151017_night.csv', '10_20151125_noon.csv', '11_20151024_night.csv', '12_20150928_noon.csv', '13_20150929_noon.csv',
            '14_20151014_night.csv','15_20151126_night.csv', '16_20151128_night.csv', '17_20150925_noon.csv', '18_20150926_noon.csv',
            '19_20151114_noon.csv', '20_20151129_night.csv', '21_20151016_noon.csv']

# features_files = ['21_20151016_noon.csv']

X_all = {} # psd
X_all_eog = {} # psd + eog
y_all = {} # datos perclos raw
y_all_limits = {} # thresholds perclos para cada experimento
y_all_class = {} # datos parseados a su clase

for experiment in features_files:
    
    ''' EOG parpadeos por epoch'''
    mat_data = read_mat(f'./SEED-VIG/Raw_Data/{experiment[:-4]}.mat')
    
    sfreq = mat_data['EOG']['eog_config']['current_sample_rate']
    samples = mat_data['EOG']['eog_h']*1e-6 
    samples = np.vstack((samples, mat_data['EOG']['eog_v']*1e-6))

    ch_names = ['EOG_H', 'EOG_V']
    ch_types = ["eog"]*len(ch_names)

    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

    info.set_montage('standard_1020')
    
    raw = mne.io.RawArray(samples, info, verbose=False)
    
    # Busqueda del treshold para el parpadeo
    # En los 5 primeros segundos del experimento, buscamos el percentil 2.5 y calculamos su valor absoluto como threshold para detectar parpadeos
    threshold = abs(np.percentile(raw.get_data()[1][:5*1000], 2.5))
    
    blinks = mne.preprocessing.find_eog_events(raw, filter_length='10s', thresh=threshold, verbose=False)
    blinks = blinks.T[0]
    
    n_samples = 885
    dur_sample = 8 # segs
    len_sample = 8*125
    
    blinks_per_sample = []
    for i in range(n_samples):
        blinks_per_sample.append([])
        
    for val in blinks:
        blinks_per_sample[val//len_sample].append(val)
    
    n_blinks_per_sample = [0]*n_samples
        
    for i in range(885):
        n_blinks_per_sample[i] = len(blinks_per_sample[i])
        
    scaler = MinMaxScaler(feature_range=(0, 1))
    n_blinks_per_sample = np.array(n_blinks_per_sample)
    n_blinks_per_sample = scaler.fit_transform(n_blinks_per_sample.reshape(-1,1))
    
    
    ''' PSD '''
    mat_data = read_mat(f'./SEED-VIG/EEG_Feature_5Bands/{experiment[:-4]}.mat')
    perclos_data = read_mat(f'./SEED-VIG/perclos_labels/{experiment[:-4]}.mat')
    y = np.array(perclos_data['perclos'])

    n_channels = 17
    psd_data = mat_data['psd_movingAve']
    perclos = []
    psd_delta = []
    psd_theta = []
    psd_alpha = []
    psd_beta = []
    psd_gamma = []
    start = 0
    end = 885
    for t in range(start, end):
        vals= np.zeros(5)

        for i in range(n_channels):
            vals[0] += psd_data[i][t][0]
            vals[1] += psd_data[i][t][1]
            vals[2] += psd_data[i][t][2]
            vals[3] += psd_data[i][t][3]
            vals[4] += psd_data[i][t][4]
        vals /= 17

        perclos.append(y[t])
        psd_delta.append(vals[0])
        psd_theta.append(vals[1])
        psd_alpha.append(vals[2])
        psd_beta.append(vals[3])
        psd_gamma.append(vals[4])

    
    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_delta = np.array(psd_delta)
    psd_delta = scaler.fit_transform(psd_delta.reshape(-1,1))

    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_theta = np.array(psd_theta)
    psd_theta = scaler.fit_transform(psd_theta.reshape(-1,1))

    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_alpha = np.array(psd_alpha)
    psd_alpha = scaler.fit_transform(psd_alpha.reshape(-1,1))

    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_beta = np.array(psd_beta)
    psd_beta = scaler.fit_transform(psd_beta.reshape(-1,1))
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    psd_gamma = np.array(psd_gamma)
    psd_gamma = scaler.fit_transform(psd_gamma.reshape(-1,1))

    X = psd_delta
    X = np.hstack((X, psd_theta))
    X = np.hstack((X, psd_alpha))
    X = np.hstack((X, psd_beta))
    ''' DESCOMENTAR PARA USAR PSD DE GAMMA '''
    X = np.hstack((X, psd_gamma))
    
    X_all[f'{experiment[:-4]}'] = X # guardamos los psd
    
    X = np.hstack((X, n_blinks_per_sample))
    
    X_all_eog[f'{experiment[:-4]}'] = X # guardamos psd + eog
    
    ''' LABELS '''
    y_all[f'{experiment[:-4]}'] = y # guardamos perclos raw
    
    range_values = (np.max(y)-np.min(y))/100
    lim_leve = np.min(y)+range_values*12.5
    lim_moderada = np.min(y)+range_values*30
    y_all_limits[f'{experiment[:-4]}'] = [lim_leve, lim_moderada]
    
    y_all_class[f'{experiment[:-4]}'] = parse_y(y, lim_leve, lim_moderada) # guardamos ya clasificada
    
print(f'Generados los diccionarios X_all e y_all')

In [ ]:
print(X_all_eog['1_20151124_noon_2'].shape)

for key in X_all:
    print(key)

# Busqueda de hyperparametros y muestra de los resultados

## Funciones para la busqueda y el plot

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import train_test_split

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from scipy.stats import uniform, randint, loguniform, gamma

def find_and_test_best_model_clust(X, y, model, space, cv=10, n_iter=100, scoring=None):
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    search = RandomizedSearchCV(model, space, n_iter=n_iter, scoring=scoring, n_jobs=-1, cv=cv, random_state=42)
    result = search.fit(X, y)
    
    print(f"Best Model:")
    print(f"\tScore: {result.best_score_:.4f}")
    print(f"\tHyperparameters: {result.best_params_}")
    
    print('\nCluestering evaluation:')
    best_model = result.best_estimator_
    final_predictions = best_model.predict(X)
    
    # The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.
    silhouette = silhouette_score(X, final_predictions)
    print(f"\tSilhouette Coefficient: {silhouette:.4f}")
    
    # The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
    calinski = calinski_harabasz_score(X, final_predictions)
    print(f"\tCalinski-Harabasz Index: {calinski:.4f}")
    
    # This index signifies the average 'similarity' between clusters, where the similarity is a measure that compares the distance between clusters with the size of the clusters themselves.
    # Zero is the lowest possible score. Values closer to zero indicate a better partition.
    davies = davies_bouldin_score(X, final_predictions)
    print(f"\tDavies Bouldin Index: {davies:.4f}")
    
    return best_model, silhouette, calinski, davies

def plot_results_clust(X, y, model, lim_leve_moderada, lim_moderada_severa):
    plt.figure(figsize=(20, 8))
    plt.axhline(y=lim_leve_moderada, color='orange', linestyle='-', zorder=0, linewidth=3)
    plt.axhline(y=lim_moderada_severa, color='red', linestyle='-', zorder=0, linewidth=3)
    plt.plot(y, c='k', alpha=0.6)
    colors = model.predict(X)
    plt.scatter(range(len(y)), y, c=colors)
    #plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

## K-Means

In [ ]:
from sklearn.cluster import KMeans

### EEG

In [ ]:
csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    X, y = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_minmax = scaler.fit_transform(X)
    
    print(f"Experiment: {key}")
    model = KMeans(random_state=42)

    space = dict()
    space['n_clusters'] = [2, 3, 4]
    space['init'] = ['k-means++', 'random']
    space['n_init'] = randint(5, 20)
    space['max_iter'] = randint(200, 400)
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['algorithm'] = ['auto', 'full', 'elkan']

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_minmax, y_all[key], model, space, cv=10, n_iter=20)
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_minmax, y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/KMeans-EEG.csv', index=False)

### PSD

In [ ]:
csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = KMeans(random_state=42)

    space = dict()
    space['n_clusters'] = [2, 3, 4]
    space['init'] = ['k-means++', 'random']
    space['n_init'] = randint(5, 20)
    space['max_iter'] = randint(200, 400)
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['algorithm'] = ['auto', 'full', 'elkan']

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all[key], y_all[key], model, space, cv=10, n_iter=20)
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/KMeans-PSD.csv', index=False)

### PSD + EOG

In [ ]:
csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = KMeans(random_state=42)

    space = dict()
    space['n_clusters'] = [2, 3, 4]
    space['init'] = ['k-means++', 'random']
    space['n_init'] = randint(5, 20)
    space['max_iter'] = randint(200, 400)
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['algorithm'] = ['auto', 'full', 'elkan']

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all_eog[key], y_all[key], model, space, cv=10, n_iter=20)
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all_eog[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/KMeans-PSD+EOG.csv', index=False)

### Resto de cosas de K-Means

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [ ]:
def kmeans_simple_plot(X, y, n_plots=3):

    fig, ax = plt.subplots(ncols=n_plots ,figsize=(20,4))

    for k in range(2, 2+n_plots):
        ax[k-2].title.set_text(f"Clusters when k={k}")

        kmeans = KMeans(n_clusters=k)
        y_pred = kmeans.fit_predict(X)

        ax[k-2].scatter(X.T[0], X.T[1], c=y_pred)
        centroids = kmeans.cluster_centers_
        ax[k-2].scatter(centroids[:, 0], centroids[:, 1], s=100, c='red')

    plt.show()

In [ ]:
def kmeans_complex_plot(X, y, n_plots=3):
    # Grafica compleja

    fig, ax = plt.subplots(nrows=n_plots, ncols=2 ,figsize=(20,15))
    fig. tight_layout(pad=5)

    for k in range(2, 2+n_plots):
        kmeans = KMeans(n_clusters=k, random_state=42)
        y_pred = kmeans.fit_predict(X)

        ### Silhouette Diagram ###

        ax[k-2][0].set_xlim([-0.1, 1])
        ax[k-2][0].set_ylim([0, len(X) + (k + 1) * 10])

        silhouette_avg = silhouette_score(X, y_pred)

        sample_silhouette_values = silhouette_samples(X, y_pred)

        y_lower = 10
        for i in range(k):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[y_pred == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / k)
            ax[k-2][0].fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax[k-2][0].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax[k-2][0].axvline(x=silhouette_avg, color="red", linestyle="--")

        ax[k-2][0].set_title(f"The silhouette diagram for n_clusters = {k}. Avg = {silhouette_avg:.3f}")
        ax[k-2][0].set_xlabel("The silhouette coefficient values")
        ax[k-2][0].set_ylabel("Cluster label")

        ax[k-2][0].set_yticks([])  # Clear the yaxis labels / ticks
        ax[k-2][0].set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        ### Clusters Visualization ###
        ax[k-2][1].set_title(f"Clusters visualization for n_clusters = {k}")
        ax[k-2][1].scatter(X.T[0], X.T[1], marker='.', s=50, lw=0, alpha=1,c=y_pred, edgecolor='k')
        centroids = kmeans.cluster_centers_
        ax[k-2][1].scatter(centroids[:, 0], centroids[:, 1], marker='o', c="white", alpha=1, s=200, edgecolor='k')
        for i, c in enumerate(centroids):
                ax[k-2][1].scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data with n_clusters from %d to %d" % (2, n_plots+1)), fontweight='bold')
    plt.show()

In [ ]:
def kmeans_predictions_plot(X, y, k=3):
    kmeans = KMeans(n_clusters=k)
    predictions = kmeans.fit_predict(X)

    plt.figure(figsize=(20, 8))
    plt.plot(y, color='gray')
    plt.scatter(range(len(predictions)), y, c=predictions)
    plt.title('Coloreado del valor PERCLOS y el cluster asociado')
    plt.xlabel('Epoch')
    plt.ylabel('% PERCLOS')
    plt.show()

In [ ]:
k_values = list(range(2, 5+1))
silhouette_scores = []

key = '11_20151024_night'

for k in k_values:
    kmeans = KMeans(n_clusters=k)
    
    kmeans.fit_predict(X_all[key])
    silhouette_scores.append(silhouette_score(X_all[key], kmeans.labels_))
    
plt.plot(k_values, silhouette_scores, '-o', label='normal')
plt.xticks(range(2,6))
plt.ylim([0, 1])
plt.title('Comparativa entre el k y el valor de Silhouette')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.legend()
plt.show()

In [ ]:
kmeans_simple_plot(X_all[key], y_all[key], 4)
kmeans_complex_plot(X_all[key], y_all[key], 4)

In [ ]:
kmeans_predictions_plot(X_all[key], y_all[key], 2)

## GM

In [ ]:
from sklearn.mixture import GaussianMixture

### EEG

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    X, y = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_minmax = scaler.fit_transform(X)
    
    
    print(f"Experiment: {key}")
    model = GaussianMixture(max_iter=200, random_state=42)

    space = dict()
    space['n_components'] = randint(2, 4)
    space['covariance_type'] = ['full', 'tied', 'diag', 'spherical']
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['reg_covar'] = [1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5]
    space['n_init'] = randint(1, 5)
    space['init_params'] = ['kmeans', 'random']
    space['warm_start'] = [True, False]

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_minmax, y_all[key], model, space, cv=10, n_iter=30)
    
    print(f'Ha convergido? {best_model.converged_}, n_its: {best_model.n_iter_}')
    print(np.round(best_model.weights_,2))
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_minmax, y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/GM-EEG.csv', index=False)

### PSD

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = GaussianMixture(max_iter=200, random_state=42)

    space = dict()
    space['n_components'] = randint(2, 4)
    space['covariance_type'] = ['full', 'tied', 'diag', 'spherical']
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['reg_covar'] = [1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5]
    space['n_init'] = randint(1, 5)
    space['init_params'] = ['kmeans', 'random']
    space['warm_start'] = [True, False]

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all[key], y_all[key], model, space, cv=10, n_iter=30)
    
    print(f'Ha convergido? {best_model.converged_}, n_its: {best_model.n_iter_}')
    print(np.round(best_model.weights_,2))
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/GM-PSD.csv', index=False)

### PSD + EOG

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = GaussianMixture(max_iter=200, random_state=42)

    space = dict()
    space['n_components'] = randint(2, 4)
    space['covariance_type'] = ['full', 'tied', 'diag', 'spherical']
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['reg_covar'] = [1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5]
    space['n_init'] = randint(1, 5)
    space['init_params'] = ['kmeans', 'random']
    space['warm_start'] = [True, False]

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all_eog[key], y_all[key], model, space, cv=10, n_iter=30)
    
    print(f'Ha convergido? {best_model.converged_}, n_its: {best_model.n_iter_}')
    print(np.round(best_model.weights_,2))
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all_eog[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/GM-PSD+EOG.csv', index=False)

### Resto de cosas de GM

In [ ]:
from sklearn.mixture import GaussianMixture
def gm_predictions_plot(X, y, n=3):
    gm = GaussianMixture(n_components=n, n_init=50)
    gm.fit(X)
    predictions = gm.predict(X)

    plt.figure(figsize=(20, 8))
    plt.plot(y, color='gray')
    plt.scatter(range(len(predictions)), y, c=predictions)
    plt.title('GMM - Coloreado del valor PERCLOS y el cluster asociado')
    plt.xlabel('Epoch')
    plt.ylabel('% PERCLOS')
    plt.show()

# BGM

In [ ]:
from sklearn.mixture import BayesianGaussianMixture

### EEG

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    X, y = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_minmax = scaler.fit_transform(X)
    
    print(f"Experiment: {key}")
    model = BayesianGaussianMixture(max_iter=200, random_state=42)

    space = dict()
    space['n_components'] = randint(2, 4)
    space['covariance_type'] = ['full', 'tied', 'diag', 'spherical']
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['reg_covar'] = [1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5]
    space['n_init'] = randint(1, 5)
    space['init_params'] = ['kmeans', 'random']
    space['weight_concentration_prior_type'] = ['dirichlet_process', 'dirichlet_distribution']
    space['warm_start'] = [True, False]

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_minmax, y_all[key], model, space, cv=10, n_iter=20)
    
    print(f'Ha convergido? {best_model.converged_}, n_its: {best_model.n_iter_}')
    print(np.round(best_model.weights_,2))
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_minmax, y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/BGM-EEG.csv', index=False)

### PSD

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = BayesianGaussianMixture(max_iter=200, random_state=42)

    space = dict()
    space['n_components'] = randint(2, 4)
    space['covariance_type'] = ['full', 'tied', 'diag', 'spherical']
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['reg_covar'] = [1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5]
    space['n_init'] = randint(1, 5)
    space['init_params'] = ['kmeans', 'random']
    space['weight_concentration_prior_type'] = ['dirichlet_process', 'dirichlet_distribution']
    space['warm_start'] = [True, False]

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all[key], y_all[key], model, space, cv=10, n_iter=20)
    
    print(f'Ha convergido? {best_model.converged_}, n_its: {best_model.n_iter_}')
    print(np.round(best_model.weights_,2))
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/BGM-PSD.csv', index=False)

### PSD + EOG

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = BayesianGaussianMixture(max_iter=200, random_state=42)

    space = dict()
    space['n_components'] = randint(2, 4)
    space['covariance_type'] = ['full', 'tied', 'diag', 'spherical']
    space['tol'] = [1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]
    space['reg_covar'] = [1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5]
    space['n_init'] = randint(1, 5)
    space['init_params'] = ['kmeans', 'random']
    space['weight_concentration_prior_type'] = ['dirichlet_process', 'dirichlet_distribution']
    space['warm_start'] = [True, False]

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all_eog[key], y_all[key], model, space, cv=10, n_iter=20)
    
    print(f'Ha convergido? {best_model.converged_}, n_its: {best_model.n_iter_}')
    print(np.round(best_model.weights_,2))
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all_eog[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/BGM-PSD+EOG.csv', index=False)

### Resto de cosas de Bayesian Gaussian Mixture

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
def bgm_predictions_plot(X, y):
    bgm = BayesianGaussianMixture(n_components=6, n_init=20, random_state=42)
    bgm.fit(X)

    print(f'Ha convergido? {bgm.converged_}, n_its: {bgm.n_iter_}')
    print(np.round(bgm.weights_,2))

    predictions = bgm.predict(X)

    plt.figure(figsize=(20, 8))
    plt.plot(y, color='gray')
    plt.scatter(range(len(predictions)), y, c=predictions)
    plt.title('BGM - Coloreado del valor PERCLOS y el cluster asociado')
    plt.xlabel('Epoch')
    plt.ylabel('% PERCLOS')
    plt.show()

# Affinity Propagation

In [ ]:
from sklearn.cluster import AffinityPropagation

### EEG

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    X, y = read_features(f'./features/{key}.csv')
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_minmax = scaler.fit_transform(X)
    
    print(f"Experiment: {key}")
    model = AffinityPropagation(max_iter=1000, random_state=42)

    space = dict()
    space['damping'] = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    space['convergence_iter'] = randint(8, 15)
    space['preference'] = range(-1, -10, -1)

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_minmax, y_all[key], model, space, cv=5, n_iter=10, scoring='homogeneity_score')
    
    cluster_centers_indices = best_model.cluster_centers_indices_
    n_clusters_ = len(cluster_centers_indices)
    print(f'Numero de Clusters: {n_clusters_}')
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_minmax, y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/AffProp-EEG.csv', index=False)

In [ ]:
key = '21_20151016_noon'


print(f"Experiment: {key}")
model = AffinityPropagation(max_iter=1000, random_state=42)

space = dict()
space['damping'] = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
space['convergence_iter'] = randint(8, 15)
space['preference'] = range(-1, -10, -1)

best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all[key], y_all[key], model, space, cv=5, n_iter=10, scoring='homogeneity_score')

cluster_centers_indices = best_model.cluster_centers_indices_
n_clusters_ = len(cluster_centers_indices)
print(f'Numero de Clusters: {n_clusters_}')

lim_leve_moderada, lim_moderada_severa = y_all_limits[key]

In [ ]:
plt.figure(figsize=(20, 8))
plt.axhline(y=lim_leve_moderada, color='orange', linestyle='-', zorder=0, linewidth=3)
plt.axhline(y=lim_moderada_severa, color='red', linestyle='-', zorder=0, linewidth=3)
plt.plot(y_all[key], c='k', alpha=0.6)
colors = best_model.predict(X_all[key])
plt.scatter(range(len(y_all[key])), y_all[key], c=colors)
size_label = 26; plt.xlabel('Epoch', fontweight ='bold', labelpad = 10,fontsize = size_label); plt.ylabel('PERCLOS', fontweight ='bold', labelpad = 10,fontsize = size_label);
size_ticks = 24; plt.xticks(fontsize=size_ticks); plt.yticks(fontsize=size_ticks);
plt.savefig("E:/UNIVERSIDAD/TFG/TRABAJO/Images-Test/NOMBRE2.pdf", bbox_inches='tight')
plt.show()

### PSD

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = AffinityPropagation(max_iter=1000, random_state=42)

    space = dict()
    space['damping'] = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    space['convergence_iter'] = randint(8, 15)
    space['preference'] = range(-1, -10, -1)

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all[key], y_all[key], model, space, cv=5, n_iter=10, scoring='homogeneity_score')
    
    cluster_centers_indices = best_model.cluster_centers_indices_
    n_clusters_ = len(cluster_centers_indices)
    print(f'Numero de Clusters: {n_clusters_}')
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/AffProp-PSD.csv', index=False)

### PSD + EOG

In [ ]:
key = '21_20151016_noon'

csv_dict = {
    'sujeto': [],
    'silhouette': [],
    'calinski': [],
    'davies': []
}

for key in X_all:
    print(f"Experiment: {key}")
    model = AffinityPropagation(max_iter=1000, random_state=42)

    space = dict()
    space['damping'] = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    space['convergence_iter'] = randint(8, 15)
    space['preference'] = range(-1, -10, -1)

    best_model, silhouette, calinski, davies = find_and_test_best_model_clust(X_all_eog[key], y_all[key], model, space, cv=5, n_iter=10, scoring='homogeneity_score')
    
    cluster_centers_indices = best_model.cluster_centers_indices_
    n_clusters_ = len(cluster_centers_indices)
    print(f'Numero de Clusters: {n_clusters_}')
    
    csv_dict['sujeto'].append(key)
    csv_dict['silhouette'].append(silhouette)
    csv_dict['calinski'].append(calinski)
    csv_dict['davies'].append(davies)

    lim_leve_moderada, lim_moderada_severa = y_all_limits[key]
    plot_results_clust(X_all_eog[key], y_all[key], best_model, lim_leve_moderada, lim_moderada_severa)
    
df = pd.DataFrame(csv_dict)
df.to_csv('./results_clust/AffProp-PSD+EOG.csv', index=False)

### Resto de cosas de Affinity Propagation

In [ ]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics

from itertools import cycle

In [ ]:
#features_files = ['4_20151105_noon.mat', '4_20151107_noon.mat', '5_20141108_noon.mat','12_20150928_noon.mat','14_20151014_night.mat', '18_20150926_noon.mat', '21_20151016_noon.mat']
key = '21_20151016_noon'

af = AffinityPropagation(preference=-6, max_iter=3000, affinity='euclidean', random_state=42)

af.fit(X_all[key])

cluster_centers_indices = af.cluster_centers_indices_
predictions = af.labels_
n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)

In [ ]:
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    class_members = predictions == k
    cluster_center = X_all[key][cluster_centers_indices[k]]
    plt.plot(X_all[key][class_members, 0], X_all[key][class_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=14)
    for x in X_all[key][class_members]:
        plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(y_all[key], color='r', alpha=0.7)
plt.scatter(range(len(predictions)), y_all[key], c=predictions)
plt.title('AP - Coloreado del valor PERCLOS y el cluster asociado')
plt.xlabel('Epoch')
plt.ylabel('% PERCLOS')
plt.show()